# Inicio Rapido

LangChain tiene varios componentes diseñados para ayudar a construir aplicaciones de preguntas y respuestas, y aplicaciones RAG de manera más general. Para familiarizarnos con ellos, construiremos una aplicación simple de preguntas y respuestas sobre una fuente de datos de texto. En el proceso, revisaremos una arquitectura típica de preguntas y respuestas, discutiremos los componentes relevantes de LangChain y resaltaremos recursos adicionales para técnicas más avanzadas de preguntas y respuestas. También veremos cómo LangSmith puede ayudarnos a rastrear y entender nuestra aplicación. LangSmith será cada vez más útil a medida que nuestra aplicación crezca en complejidad.

Arquitectura
Crearemos una aplicación RAG típica según se describe en la introducción de preguntas y respuestas, que consta de dos componentes principales:

1. Indexación: una canalización para ingerir datos desde una fuente e indexarlos. Esto suele ocurrir offline.

2. Recuperación y generación: la cadena RAG real, que toma la consulta del usuario en tiempo de ejecución y recupera los datos relevantes del índice, luego los pasa al modelo.

La secuencia completa, desde datos en bruto hasta la respuesta, se verá así:

**Indexación**
- Carga: Primero necesitamos cargar nuestros datos. Utilizaremos DocumentLoaders para esto.
- División: Los separadores de texto dividen documentos grandes en fragmentos más pequeños. Esto es útil tanto para indexar datos como para pasarlos a un modelo, ya que los fragmentos grandes son más difíciles de buscar y no caben en la ventana de contexto finito de un modelo.
- Almacenamiento: Necesitamos un lugar para almacenar e indexar nuestros fragmentos, para que puedan buscarse posteriormente. Esto se hace a menudo utilizando un VectorStore y un modelo de embeddings.

**Recuperación y generación**
- Recuperación: Dada una entrada del usuario, se recuperan los fragmentos relevantes del almacenamiento utilizando un Recuperador.
- Generación: Un ChatModel / LLM produce una respuesta utilizando un prompt que incluye la pregunta y los datos recuperados.

Configuración
Dependencias
Utilizaremos un modelo de chat de OpenAI, embeddings y un vector store Chroma en este tutorial, pero todo lo mostrado aquí funciona con cualquier modelo de chat o LLM, embeddings, y vector store o recuperador.

Utilizaremos los siguientes paquetes:

In [1]:
# %pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

In [2]:
import dotenv

dotenv.load_dotenv()

True

Necesitamos configurar la variable de entorno OPENAI_API_KEY, que se puede hacer directamente o cargar desde un archivo .env de esta manera:

# LangSmith

Muchas de las aplicaciones que construyas con LangChain contendrán múltiples pasos con múltiples invocaciones de llamadas a LLM. A medida que estas aplicaciones se vuelven más complejas, se vuelve crucial poder inspeccionar qué está sucediendo exactamente dentro de tu cadena o agente. La mejor manera de hacer esto es con LangSmith.

Cabe destacar que LangSmith no es necesario, pero es útil. Si deseas utilizar LangSmith, después de registrarte en el enlace anterior, asegúrate de configurar tus variables de entorno para comenzar a registrar trazas:

# Vista previa

En esta guía, construiremos una aplicación de preguntas y respuestas sobre la publicación del blog "LLM Powered Autonomous Agents" de Lilian Weng, lo que nos permitirá hacer preguntas sobre el contenido de la publicación.

Podemos crear un simple flujo de trabajo de indexación y una cadena RAG para lograr esto en ~20 líneas de código:

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [4]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [5]:
rag_chain.invoke("¿Que es la descomposición de tareas?")

'La descomposición de tareas es un proceso en el que se divide un problema o tarea complicada en pasos más pequeños y manejables. Esto permite que un agente o modelo de IA pueda abordar la tarea de manera más efectiva al enfocarse en cada paso por separado. Se pueden utilizar diferentes técnicas, como el uso de instrucciones específicas o la generación de múltiples pensamientos por paso, para llevar a cabo la descomposición de tareas.'

In [6]:
# cleanup
vectorstore.delete_collection()

# Paseo detallado

Vamos a revisar el código anterior paso a paso para comprender realmente lo que está sucediendo.

# 1. Indexación: Carga
Primero, necesitamos cargar el contenido del blog. Podemos utilizar DocumentLoaders para esto, que son objetos que cargan datos desde una fuente y devuelven una lista de Documentos. Un Documento es un objeto con algún page_content (str) y metadata (dict).

En este caso, utilizaremos WebBaseLoader, que utiliza urllib para cargar HTML desde URL web y BeautifulSoup para analizarlo a texto. Podemos personalizar la conversión de HTML a texto pasando parámetros al analizador BeautifulSoup a través de bs_kwargs (consultar la documentación de BeautifulSoup). En este caso, solo son relevantes las etiquetas HTML con las clases "post-content", "post-title" o "post-header", por lo que eliminaremos todas las demás.

In [7]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [8]:
len(docs[0].page_content)

42824

In [9]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


## Adentrémonos más

**DocumentLoader:** Objeto que carga datos desde una fuente como una lista de Documentos.
- **Documentación:** Documentación detallada sobre cómo utilizar DocumentLoaders.
- **Integraciones:** Más de 160 integraciones para elegir.
- **Interfaz:** Referencia de API para la interfaz base.

# 2. Indexación: División

Nuestro documento cargado tiene más de 42,000 caracteres de longitud. Esto es demasiado largo para ajustarse en la ventana de contexto de muchos modelos. Incluso para aquellos modelos que podrían ajustar la publicación completa en su ventana de contexto, los modelos pueden tener dificultades para encontrar información en entradas muy largas.

Para manejar esto, dividiremos el Documento en fragmentos para la inserción de vectores. Esto debería ayudarnos a recuperar solo las partes más relevantes del blog en tiempo de ejecución.

En este caso, dividiremos nuestros documentos en fragmentos de 1000 caracteres con 200 caracteres de superposición entre fragmentos. La superposición ayuda a mitigar la posibilidad de separar una declaración de un contexto importante relacionado con ella. Utilizamos RecursiveCharacterTextSplitter, que dividirá recursivamente el documento utilizando separadores comunes como nuevas líneas hasta que cada fragmento tenga el tamaño adecuado. Este es el separador de texto recomendado para casos de uso de texto genérico.

Configuramos add_start_index=True para que el índice de caracteres en el que comienza cada Documento dividido dentro del Documento inicial se conserve como atributo de metadatos "start_index".

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [11]:
len(all_splits)

66

In [12]:
len(all_splits[0].page_content)

969

In [13]:
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

## Adentrémonos más

**TextSplitter:** Objeto que divide una lista de Documentos en fragmentos más pequeños. Subclase de DocumentTransformers.
- **Explora Separadores conscientes del contexto, que mantienen la ubicación ("contexto") de cada fragmento en el Documento original:**
  - [Archivos Markdown](https://python.langchain.com/docs/modules/data_connection/document_transformers/markdown_header_metadata)

  - [Código (py o js)](https://python.langchain.com/docs/integrations/document_loaders/source_code)
  - [Documentos científicos](https://python.langchain.com/docs/integrations/document_loaders/grobid)
- **[Interfaz](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.TextSplitter.html):** Referencia de API para la interfaz base.

**DocumentTransformer:** Objeto que realiza una transformación en una lista de Documentos.
- **Documentación:** Documentación detallada sobre cómo utilizar DocumentTransformers.
- **Integraciones:** Integraciones disponibles.
- **Interfaz:** Referencia de API para la interfaz base.

# 3. Indexación: Almacenamiento

Ahora necesitamos indexar nuestros 66 fragmentos de texto para que podamos buscar en ellos en tiempo de ejecución. La forma más común de hacer esto es incrustar el contenido de cada fragmento de documento e insertar estas incrustaciones en una base de datos de vectores (o vector store). Cuando queremos buscar en nuestros fragmentos, tomamos una consulta de búsqueda de texto, la incrustamos y realizamos algún tipo de búsqueda de "similitud" para identificar los fragmentos almacenados con las incrustaciones más similares a nuestra incrustación de consulta. La medida de similitud más simple es la similitud coseno; medimos el coseno del ángulo entre cada par de incrustaciones (que son vectores de alta dimensión).

Podemos incrustar y almacenar todos nuestros fragmentos de documentos en un solo comando utilizando el vector store Chroma y el modelo [OpenAIEmbeddings](https://python.langchain.com/docs/integrations/text_embedding/openai).

In [14]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

## Adentrémonos más

**Embeddings:** Envoltura alrededor de un modelo de incrustación de texto, utilizado para convertir texto en incrustaciones.
- **Documentación:** Documentación detallada sobre cómo utilizar incrustaciones.
- **Integraciones:** Más de 30 integraciones para elegir.
- **Interfaz:** Referencia de API para la interfaz base.

**VectorStore:** Envoltura alrededor de una base de datos de vectores, utilizado para almacenar y consultar incrustaciones.
- **Documentación:** Documentación detallada sobre cómo utilizar vector stores.
- **Integraciones:** Más de 40 integraciones para elegir.
- **Interfaz:** Referencia de API para la interfaz base.

Esto completa la parte de Indexación del flujo de trabajo. En este punto, tenemos un vector store consultable que contiene los contenidos fragmentados de nuestra publicación de blog. Dada una pregunta del usuario, idealmente deberíamos poder devolver los fragmentos de la publicación de blog que responden a la pregunta.

# 4. Recuperación y Generación: Recuperar

Ahora escribamos la lógica real de la aplicación. Queremos crear una aplicación sencilla que tome una pregunta del usuario, busque documentos relevantes para esa pregunta, pase los documentos recuperados y la pregunta inicial a un modelo y devuelva una respuesta.

Primero, necesitamos definir nuestra lógica para buscar documentos. LangChain define una interfaz de Retriever que envuelve un índice que puede devolver Documentos relevantes dada una consulta de cadena.

El tipo más común de Retriever es VectorStoreRetriever, que utiliza las capacidades de búsqueda de similitud de un vector store para facilitar la recuperación. Cualquier VectorStore se puede convertir fácilmente en un Retriever con VectorStore.as_retriever():

In [15]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [16]:
retrieved_docs = retriever.invoke("¿Cuáles son los enfoques para la descomposición de tareas?")

In [17]:
len(retrieved_docs)

6

In [18]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


## Adentrémonos más

Los vector stores se utilizan comúnmente para la recuperación, pero también hay otras formas de hacer recuperación.

**Retriever:** Un objeto que devuelve Documentos dado una consulta de texto.

- **Documentación:** Documentación adicional sobre la interfaz y las técnicas de recuperación incorporadas. Algunas de ellas incluyen:
  - MultiQueryRetriever genera variantes de la pregunta de entrada para mejorar la tasa de éxito de la recuperación.
  - MultiVectorRetriever (diagrama a continuación) en cambio genera variantes de las incrustaciones, también para mejorar la tasa de éxito de la recuperación.
  - Max marginal relevance selecciona relevancia y diversidad entre los documentos recuperados para evitar pasar contexto duplicado.
  - Los documentos pueden filtrarse durante la recuperación del vector store mediante filtros de metadatos, como con un Self Query Retriever.

- **Integraciones:** Integraciones con servicios de recuperación.
  
- **Interfaz:** Referencia de API para la interfaz base.

# 5. Recuperación y Generación: Generar

Vamos a ponerlo todo junto en una cadena que toma una pregunta, recupera documentos relevantes, construye un indicador, lo pasa a un modelo y analiza la salida.

Utilizaremos el modelo de chat de OpenAI gpt-3.5-turbo, pero se podría sustituir por cualquier LangChain LLM o ChatModel.

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

Usaremos un mensaje para RAG que se registra en el centro de mensajes de LangChain (aquí).

In [20]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [21]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [22]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


Utilizaremos el protocolo LCEL Runnable para definir la cadena, lo que nos permitirá:
- conectar componentes y funciones de manera transparente,
- rastrear automáticamente nuestra cadena en LangSmith,
- obtener llamadas en streaming, asíncronas y por lotes directamente.

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
for chunk in rag_chain.stream("¿Qué es la descomposición de tareas?"):
    print(chunk, end="", flush=True)

La descomposición de tareas es el proceso de dividir una tarea compleja en pasos más pequeños y manejables. Esto permite abordar la tarea de manera más eficiente y facilita la planificación y ejecución de cada paso. La descomposición de tareas puede realizarse mediante instrucciones específicas, como en el caso de escribir un esquema de una historia, o mediante la interacción con un modelo de IA.

Consulte el rastro de [LangSmith]()

## Adentrémonos más

**Choosing a model:**

- **ChatModel:** Un modelo de chat respaldado por LLM. Toma una secuencia de mensajes y devuelve un mensaje.
  - **Documentación:** Documentación detallada.
  - **Integraciones:** Más de 25 integraciones para elegir.
  - **Interfaz:** Referencia de API para la interfaz base.

- **LLM:** Un modelo de lenguaje que toma un texto de entrada y devuelve un texto de salida.
  - **Documentación:** Documentación detallada.
  - **Integraciones:** Más de 75 integraciones para elegir.
  - **Interfaz:** Referencia de API para la interfaz base.

Vea una guía sobre RAG con modelos que se ejecutan localmente aquí.

**Personalización del indicador:**

Como se muestra anteriormente, podemos cargar indicadores (por ejemplo, este indicador RAG) desde el centro de indicadores. El indicador también se puede personalizar fácilmente:

In [28]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("¿Que es la descomposicion de tareas?")

'La descomposición de tareas es un proceso en el que se divide una tarea compleja en pasos más pequeños y manejables. Esto permite a un agente o modelo abordar la tarea de manera más efectiva al enfocarse en cada paso individualmente. Gracias por preguntar!'

# Próximos pasos

Hemos cubierto mucho contenido en poco tiempo. Hay muchas funciones, integraciones y extensiones para explorar en cada una de las secciones anteriores. Además de las fuentes para profundizar mencionadas anteriormente, buenos próximos pasos incluyen:

- **Fuentes de retorno:** Aprender a devolver documentos fuente.
- **Streaming:** Aprender a transmitir salidas y pasos intermedios.
- **Agregar historial de chat:** Aprender a agregar historial de chat a tu aplicación.